In [70]:
from pathlib import Path
import pandas as pd
import numpy as np

data_dir = Path('/Users/bensnyder/Projects/plots/data/dummy_datasets/')

dfs = {df.name: pd.read_excel(df.as_posix()) for df in data_dir.glob('df*')}

In [71]:
df1 = dfs['df.xlsx'].groupby(['revr_id', 'revr_last_name', 'revr_frst_name']) \
.agg({'prop_id': lambda x: len(np.unique(x)),
'rev_stts_code': lambda x: (x=='R').astype(int).sum()/len(x),
'rev_sent_date': lambda x: pd.to_datetime(x, format="%Y-%m-%d %H:%M:%S:%f").max()}).reset_index()\
.merge(dfs['df.xlsx'].loc[dfs['df.xlsx']['rev_stts_code']=='R'].groupby('revr_id')\
.agg({'rev_sent_date': lambda x: pd.to_datetime(x, format="%Y-%m-%d %H:%M:%S:%f").max()})\
.reset_index().rename(columns={'rev_sent_date':'last_review'}), on='revr_id')\
.rename(columns={'prop_id':'request_count', 'rev_stts_code': 'review_rate', 'rev_sent_date': 'last_request'})

In [72]:
df1

,revr_id,revr_last_name,revr_frst_name,request_count,review_rate,last_request,last_review
0,A12345,Lewis,Allison,4,0.250,2019-02-13,2019-02-13
1,B56789,Snyder,Ben,8,0.625,2020-02-13,2019-02-13
2,C10111,Smith,Jenn,2,0.500,2018-02-13,2001-02-13


In [73]:
# one at a time, with more details. first, lambda functions
# lambda is a way to write a function on one line, and can be passed as an argument in another function
# for example, a lambda function that squares its inputs
squarer = lambda x: x**2
squarer(9)

81

In [82]:
# take the first dataframe out of the dictionary for simplicity
df = dfs['df.xlsx']
df

,revr_id,prop_id,rev_stts_code,rev_stts_txt,pgm_ele_code,revr_last_name,revr_frst_name,revr_emai_addr,rev_sent_date,gend_code,inst_id
0,A12345,1,R,Reviewed,NaN,Lewis,Allison,allisonlewis3@gmail.com,2019-02-13 00:00:00:000,F,3495874395
1,A12345,2,N,No Response,NaN,Lewis,Allison,allisonlewis3@gmail.com,2018-02-13 00:00:00:000,F,3495874395
2,A12345,3,N,No Response,NaN,Lewis,Allison,allisonlewis3@gmail.com,2007-02-13 00:00:00:000,F,3495874395
3,A12345,4,D,Declined to Review,NaN,Lewis,Allison,allisonlewis3@gmail.com,2001-02-13 00:00:00:000,F,3495874395
4,B56789,5,N,No Response,NaN,Snyder,Ben,snyder@gmail.com,2020-02-13 00:00:00:000,M,4598749587
5,B56789,6,R,Reviewed,NaN,Snyder,Ben,snyder@gmail.com,2019-02-13 00:00:00:000,M,4598749587
6,B56789,7,R,Reviewed,NaN,Snyder,Ben,snyder@gmail.com,2018-02-13 00:00:00:000,M,4598749587
7,B56789,8,R,Reviewed,NaN,Snyder,Ben,snyder@gmail.com,2017-02-13 00:00:00:000,M,4598749587
8,B56789,9,R,Reviewed,NaN,Snyder,Ben,snyder@gmail.com,2016-02-13 00:00:00:000,M,4598749587
9,B56789,10,C,Conflict of Interest,NaN,Snyder,Ben,snyder@gmail.com,2015-02-13 00:00:00:000,M,4598749587


In [83]:
# in the context of a groupby, lambda can be used to write very specific groupby methods. 
# each input x becomes an array. for example, say we want to get the max prop_id value
# for each reviewer (yes this can just be done with .max() but let's write our own)
# if we use df.groupby('revr_id').agg({'prop_id': lambda x: x.max()})
# each time the lambda function is called, it will get an array of all prop_id values
# for each revr_id, so [1,2,3,4] for A12345, [5,6,7,8,9,10,11,12] for B56789 and so on.
df.groupby('revr_id').agg({'prop_id': lambda x: x.max()})

,prop_id
revr_id,
A12345,4
B56789,12
C10111,14


In [86]:
# so if we want to get the count of all unique values, we can get the list of unique values
# using the numpy unique function, then take the length of that list
df1 = df.groupby('revr_id').agg({'prop_id': lambda x: len(np.unique(x))})
df1

,prop_id
revr_id,
A12345,4
B56789,8
C10111,2


In [87]:
# now we want to get the proportion of rev_stts_code that is R for each reviewer
# we can create a boolean array for which is R by using
df['rev_stts_code']=='R'

0      True
1     False
2     False
3     False
4     False
5      True
6      True
7      True
8      True
9     False
10    False
11     True
12     True
13    False
Name: rev_stts_code, dtype: bool

In [88]:
# this can be converted to int by
(df['rev_stts_code']=='R').astype(int)

0     1
1     0
2     0
3     0
4     0
5     1
6     1
7     1
8     1
9     0
10    0
11    1
12    1
13    0
Name: rev_stts_code, dtype: int64

In [90]:
# and then get the proportion by taking the sum and dividing by the length
(df['rev_stts_code']=='R').astype(int).sum()/len(df['rev_stts_code'])

0.5

In [92]:
# and in the form of a lambda for groupby
df2 = df.groupby('revr_id').agg({'rev_stts_code': lambda x: (x=='R').astype(int).sum()/len(x)})
df2

,rev_stts_code
revr_id,
A12345,0.250
B56789,0.625
C10111,0.500


In [94]:
# in order to get the max date, we first need to convert the rev_sent_date
# into a datetime column
# this can be done with the pd.to_datetime function
# often this can automatically detect the format of the date, but in this case
# having the hour minute second portion throws off the formatting, so it needs
# to be done manually
pd.to_datetime(df['rev_sent_date'], format="%Y-%m-%d %H:%M:%S:%f")

0    2019-02-13
1    2018-02-13
2    2007-02-13
3    2001-02-13
4    2020-02-13
5    2019-02-13
6    2018-02-13
7    2017-02-13
8    2016-02-13
9    2015-02-13
10   2014-02-13
11   2013-02-13
12   2001-02-13
13   2018-02-13
Name: rev_sent_date, dtype: datetime64[ns]

In [95]:
# once we have that, we can take the max date
pd.to_datetime(df['rev_sent_date'], format="%Y-%m-%d %H:%M:%S:%f").max()

Timestamp('2020-02-13 00:00:00')

In [98]:
# and again in lambda
df3 = df.groupby('revr_id').agg({'rev_sent_date': lambda x: pd.to_datetime(x, format="%Y-%m-%d %H:%M:%S:%f").max()})
df3

,rev_sent_date
revr_id,
A12345,2019-02-13
B56789,2020-02-13
C10111,2018-02-13


In [99]:
# finally, we need to get the last review date
# we first subset the data into row where rev_stts_code is R


,revr_id,prop_id,rev_stts_code,rev_stts_txt,pgm_ele_code,revr_last_name,revr_frst_name,revr_emai_addr,rev_sent_date,gend_code,inst_id
0,A12345,1,R,Reviewed,NaN,Lewis,Allison,allisonlewis3@gmail.com,2019-02-13 00:00:00:000,F,3495874395
5,B56789,6,R,Reviewed,NaN,Snyder,Ben,snyder@gmail.com,2019-02-13 00:00:00:000,M,4598749587
6,B56789,7,R,Reviewed,NaN,Snyder,Ben,snyder@gmail.com,2018-02-13 00:00:00:000,M,4598749587
7,B56789,8,R,Reviewed,NaN,Snyder,Ben,snyder@gmail.com,2017-02-13 00:00:00:000,M,4598749587
8,B56789,9,R,Reviewed,NaN,Snyder,Ben,snyder@gmail.com,2016-02-13 00:00:00:000,M,4598749587
11,B56789,12,R,Reviewed,NaN,Snyder,Ben,snyder@gmail.com,2013-02-13 00:00:00:000,M,4598749587
12,C10111,13,R,Reviewed,NaN,Smith,Jenn,smith@gmail.com,2001-02-13 00:00:00:000,F,3495874395


In [101]:
# then we run the same groupby as above to get the maximum date
df4 = df.loc[df['rev_stts_code']=='R'].groupby('revr_id').agg({'rev_sent_date': lambda x: pd.to_datetime(x, format="%Y-%m-%d %H:%M:%S:%f").max()})
df4

,rev_sent_date
revr_id,
A12345,2019-02-13
B56789,2019-02-13
C10111,2001-02-13


In [103]:
# now we need to put everything together. first, let's run all those groupbys again to add in reviewer name
df1 = df.groupby(['revr_id', 'revr_last_name', 'revr_frst_name']).agg({'prop_id': lambda x: len(np.unique(x))})

df2 = df.groupby(['revr_id', 'revr_last_name', 'revr_frst_name']).agg({'rev_stts_code': lambda x: (x=='R').astype(int).sum()/len(x)})

df3 = df.groupby(['revr_id', 'revr_last_name', 'revr_frst_name']).agg({'rev_sent_date': lambda x: pd.to_datetime(x, format="%Y-%m-%d %H:%M:%S:%f").max()})

df4 = df.loc[df['rev_stts_code']=='R'].groupby(['revr_id', 'revr_last_name', 'revr_frst_name']).agg({'rev_sent_date': lambda x: pd.to_datetime(x, format="%Y-%m-%d %H:%M:%S:%f").max()})



In [109]:
# we also want to change some of the variable names
df1 = df1.reset_index().rename(columns={'prop_id': 'request_count'})
df2 = df2.reset_index().rename(columns={'rev_stts_code': 'review_rate'})
df3 = df3.reset_index().rename(columns={'rev_sent_date': 'last_request'})
df4 = df4.reset_index().rename(columns={'rev_sent_date': 'last_review'})

In [112]:
# and now merge them together
df1 = df1.merge(df2, on=['revr_id', 'revr_last_name', 'revr_frst_name'])
df1 = df1.merge(df3, on=['revr_id', 'revr_last_name', 'revr_frst_name'])
df1 = df1.merge(df4, on=['revr_id', 'revr_last_name', 'revr_frst_name'])

In [113]:
df1

,revr_id,revr_last_name,revr_frst_name,request_count,review_rate,last_request,last_review
0,A12345,Lewis,Allison,4,0.250,2019-02-13,2019-02-13
1,B56789,Snyder,Ben,8,0.625,2020-02-13,2019-02-13
2,C10111,Smith,Jenn,2,0.500,2018-02-13,2001-02-13
